## Realizando instalações e importações de libs

In [42]:
!pip install -q -U google-generativeai

In [43]:
import google.generativeai as genai
from google.colab import userdata
import requests
import random
import json

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

## Buscando mensalidades com vencimentos próximos

In [44]:
# Busca os vencimentos próximos ou passados para criar uma interação mais rica com os usúarios.
# response_past_due_payments = requests.get("https://raw.githubusercontent.com/tiagosabadini/desafio-alura-google/main/vencimentos_proximos.json")
past_due_payments = json.loads('[{"nome":"Renan Benício Renan Araújo","vencimento":"08/05/2024","forma_pagamento":"cartão de crédito"},{"nome":"Manoel Kauê Ricardo Pereira","vencimento":"15/05/2024","forma_pagamento":"boleto"},{"nome":"Letícia Patrícia da Rocha","vencimento":"01/05/2024","forma_pagamento":"boleto"},{"nome":"Priscila Sarah Teresinha Rocha","vencimento":"10/05/2024","forma_pagamento":"pix"}]')
random_student = random.choice(past_due_payments)

# past_due_payments = []
# if resultado.status_code == 200:
#     past_due_payments = json.loads(response_past_due_payments.content)
# else:
#     print(f"Erro ao obter o JSON: {response_past_due_payments.status_code}")



## Configurando o contexto da conversa
Aqui o sistema configura como será o contexto da conversa.

In [45]:
import datetime
import json

# configure estas variáveis abaixo para alterar o comportamento do sistema

# data que o modelo vai considerar como corrente
data_atual = "09/05/2024"

# definindo o tipo de interação
data_atual_formatada = datetime.datetime.strptime(data_atual, "%d/%m/%Y")
data_vencimento_formatada = datetime.datetime.strptime(random_student['vencimento'], "%d/%m/%Y")

# vencido, a vencer, vence hoje
if data_atual_formatada < data_vencimento_formatada:
  tipo_interacao = "a vencer"
elif data_atual_formatada == data_vencimento_formatada:
  tipo_interacao = "vence hoje"
else:
  tipo_interacao = "vencido"


config = {
    "tipo_interacao": tipo_interacao,
    "nome": random_student['nome'].split()[0],
    "forma_pagamento": random_student['forma_pagamento'],
    "vencimento": random_student['vencimento'],
    "data_atual": data_atual
}

first_template_message = ""
if config["tipo_interacao"] == "vencido":
  first_template_message = f"Olá {config['nome']}, sua mensalidade venceu no dia {config['vencimento']} e não identificamos seu pagamento. Poderia nos enviar o comprovante para validarmos no nosso sistema?"
elif config["tipo_interacao"] == "vence hoje":
  first_template_message = f"Olá {config['nome']}, o vencimento da sua mensalidade está marcado para hoje. Lembre-se que o pagamento em dia te dá direito a 5% de desconto."
else:
  first_template_message = f"Olá {config['nome']}, o vencimento da sua mensalidade está próximo. Lembre-se que o pagamento em dia te dá direito a 5% de desconto."

print("Contexto definido:")
print(json.dumps(config, indent=6))
print("Primeira mensagem do sistema: ")
print(first_template_message)


Contexto definido:
{
      "tipo_interacao": "vencido",
      "nome": "Renan",
      "forma_pagamento": "cart\u00e3o de cr\u00e9dito",
      "vencimento": "08/05/2024",
      "data_atual": "09/05/2024"
}
Primeira mensagem do sistema: 
Olá Renan, sua mensalidade venceu no dia 08/05/2024 e não identificamos seu pagamento. Poderia nos enviar o comprovante para validarmos no nosso sistema?


## Configuração do Modelo

In [46]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

# Aqui estou buscando as instruções iniciais no repo do github
# para que o modelo se comporte da forma como eu preciso.
# response_system_instruction=requests.get("https://raw.githubusercontent.com/tiagosabadini/desafio-alura-google/main/system_instruction.json")
response_system_instruction=json.loads('{"system_instruction":"Você é um agente de cobrança muito educado, que representa um aplicativo chamado Tá Pago!. Seu nome é Tiago. Os educadores físicos contratam o Tá Pago! para lembrar os seus alunos de fazerem os pagamentos na data certa. O Tá Pago! avisa os alunos, recolhe os comprovantes e confere se o educador físico recebeu os pagamentos e confirma com os alunos se o pagamento foi confirmado ou não. O Tá Pago! oferece 3 formas de pagamento: cartão de crédito, pix e boleto. Para ajudar os alunos a realizarem os pagamento considere isso: * Quando o pagamento for com cartão de crédito, ofereça o link do nosso site para inserir os dados do cartão de forma segura: https://parceirodetreino.com/checkout; * Quando o pagamento for com boleto, ofereça o link para gerar um novo boleto: https://parceirodetreino.com/gerar-boleto; * Quando o pagamento for por pix, informe a chave pix: parceiro@parceirodetreino.com e solicite que o aluno nos envie o comprovante. Sempre pergunte ao aluno se ele precisa de ajuda em algo relacionado a cobrança. E ao final de toda conversa pergunte se você pode ajudar em algo mais. Auxilie o aluno a enviar o comprovante sempre pelo nosso site no link https://parceirodetreino.com/enviar-comprovante"}')
system_instruction = response_system_instruction["system_instruction"]
system_instruction = system_instruction+f" Você está falando com {config['nome']}, a forma de pagamento escolhida pelo aluno é {config['forma_pagamento']} e o vencimento está previsto para {config['vencimento']} e considere que hoje é dia {config['data_atual']}"

# if response_system_instruction.status_code == 200:
#     system_instruction = json.loads(response_system_instruction.content)
#     system_instruction = system_instruction["system_instruction"]
# else:
#     system_instruction=""

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings,system_instruction=system_instruction)

## Inicializando o Chat

In [47]:
history = [];
chat = model.start_chat(history=history)

In [ ]:
print("## Tá Pago! - Iniciando conversa ##")
print("---------------------------------------------------------------------\n\n")
print("> Tá Pago! - Tiago")
print(first_template_message)
print("---------------------------------------------------------------------\n\n")

print("> " + config['nome'])
resposta = input("Resposta\n")
while resposta != "Tá Pago!":
  response_ia = chat.send_message(resposta)
  print("> Tá Pago! - Tiago\n")
  print(response_ia.text)
  print("---------------------------------------------------------------------\n\n")
  print("> " + config['nome'])
  resposta = input("Digite Tá Pago! para finalizar, ou continue conversando\n")
  print("---------------------------------------------------------------------\n\n")

print("Chat finalizado")